# Quantum Alcubierre Warp Drive

The Alcubierre drive is a speculative warp drive idea according to which a spacecraft could achieve apparent faster-than-light travel by contracting space in front of it and expanding space behind it, under the assumption that a configurable energy-density field lower than that of vacuum (that is, negative mass) could be created.

![My Image](1.jfif)

Work based on whitepaper from https://arxiv.org/abs/gr-qc/0009013

Coded by Sam Rahmeh & Y3TI

#### Load Libraries

In [ ]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pyqubo import Array, Constraint
import dynex 

#### Setup the Alcubierre Warp Drive Parameters

In [ ]:
xm, xM = 1.0, 8.0
pm, pM = -4.0, 4.0
numPts = 90
xsm, xsM = 1.0, 10.0 
numSTEPS = 10

### Building the Alcubierre Warp Drive

As you know, the Alcubierre Warp Drive is based on:

#### 1. The Metric Tensor

The metric for the Alcubierre Warp Drive is given by:

$$
ds^2 = -c^2 dt^2 + (dx - v_s f(r_s) dt)^2 + dy^2 + dz^2
$$

where:
$c$ is the speed of light.
$ds$ is the spacetime interval.
$ v_s $ is the speed of the warp bubble.
$ f(r_s) $ is a smooth function that describes the shape of the warp bubble.
$ r_s = \sqrt{(x - x_s(t))^2 + y^2 + z^2} $ is the radial distance from the center of the bubble.
$ x_s(t) $ is the position of the center of the warp bubble as a function of time.

#### 2. Energy Density

The energy density required for the Alcubierre Warp Drive is given by:

$$
\rho = \frac{c^2}{8\pi G} \left( \frac{v_s^2}{4r_s^2} \left( \frac{\partial f}{\partial r_s} \right)^2 + \frac{v_s^2}{2r_s} \frac{\partial^2 f}{\partial r_s^2} \right)
$$

where:
$ G $ is the gravitational constant.

## Now in order to build that in **Quantum-based** solution, I rebuilt the equations as follow:

#### 1. Radial Distance Function

$$
r_s(x, \rho, x_s) = \sqrt{(x - x_s)^2 + \rho^2}
$$

where:
- $ x $ is the position coordinate.
- $ x_s $ is the position of the warp bubble center.
- $ \rho = \sqrt{y^2 + z^2} $.

#### 2. Warp Field Gradient

The warp field gradient $ \frac{df}{dr_s} $ is:

$$
\begin{aligned}
\frac{df}{dr_s} &= -\frac{1}{2} \left[ \sigma \left( \tanh^2(\sigma (R + r_s)) - 1 \right) - \frac{\sigma \left( \tanh^2(-\sigma (R - r_s)) - 1 \right)}{\tanh(\sigma R)} \right] \\
&\quad + \frac{1}{4} \left[ \sigma^2 (R + r_s) \text{sech}^2(\sigma (R + r_s)) \tanh(\sigma (R + r_s)) - \frac{\sigma^2 (R - r_s) \text{sech}^2(\sigma (R - r_s)) \tanh(\sigma (R - r_s))}{\tanh(\sigma R)} \right] \\
&\quad - \frac{1}{8} \left[ \sigma^3 (R + r_s)^2 \text{sech}^4(\sigma (R + r_s)) - \frac{\sigma^3 (R - r_s)^2 \text{sech}^4(\sigma (R - r_s))}{\tanh^2(\sigma R)} \right] \\
&\quad + \frac{1}{16} \left[ \sigma^4 (R + r_s)^3 \text{sech}^6(\sigma (R + r_s)) - \frac{\sigma^4 (R - r_s)^3 \text{sech}^6(\sigma (R - r_s))}{\tanh^3(\sigma R)} \right] \\
&\quad + \frac{1}{32} \left[ \sigma^5 (R + r_s)^4 \text{sech}^8(\sigma (R + r_s)) - \frac{\sigma^5 (R - r_s)^4 \text{sech}^8(\sigma (R - r_s))}{\tanh^4(\sigma R)} \right]
\end{aligned}
$$

where:
- $ \sigma $ is a parameter controlling the steepness of the warp bubble.
- $ R $ is the radius of the warp bubble.
- $ r_s $ is the radial distance.

#### 3. The Hamiltonian Energy Term

The Hamiltonian energy term $ H $ in the BQM is something like this:

$$
\begin{aligned}
H &= \sum_{i=1}^{N} \sum_{j=1}^{N} \sum_{k=1}^{M} \theta_{ijk} \, x_i \, p_j \, x_s^k \\
&\quad + \lambda_1 \sum_{i=1}^{N} \sum_{j=1}^{N} \sum_{k=1}^{M} \left( \alpha_{ijk} \, x_i^2 \, p_j^2 \, x_s^k + \beta_{ijk} \, x_i \, p_j^3 \, x_s^2 \right) \\
&\quad + \lambda_2 \sum_{i=1}^{N} \sum_{j=1}^{N} \sum_{k=1}^{M} \left( \gamma_{ijk} \, x_i^3 \, p_j \, x_s^2 + \delta_{ijk} \, x_i \, p_j \, x_s^3 \right) \\
&\quad + \sum_{\text{constraints}} \left[ \mu_1 \sum_{i=1}^{N} \left( \sum_{j=1}^{N} x_i p_j - \sum_{k=1}^{M} x_s^k \right)^2 + \mu_2 \sum_{j=1}^{N} \left( \sum_{i=1}^{N} p_j x_i - \sum_{k=1}^{M} x_s^k \right)^2 \right] \\
&\quad + \nu \sum_{i=1}^{N} \sum_{j=1}^{N} \sum_{k=1}^{M} \left( \frac{(x_i - p_j)^2}{x_s^k + 1} + \frac{(x_s^k - x_i)^2}{p_j + 1} \right) \\
&\quad + \omega \sum_{i=1}^{N} \sum_{j=1}^{N} \sum_{k=1}^{M} \left( \frac{x_i \, p_j \, x_s^k}{(x_i + 1)(p_j + 1)(x_s^k + 1)} \right)
\end{aligned}
$$

where:
- $ \theta_{ijk} = v_s \frac{(x - x_s)}{r_s} \frac{df}{dr_s} $
- $ \alpha_{ijk}, \beta_{ijk}, \gamma_{ijk}, \delta_{ijk} $ are higher-order coefficients for interactions.
- $ \lambda_1, \lambda_2, \mu_1, \mu_2, \nu, \omega $ are weighting factors for constraints and terms.
- $ x, p, x_s $ are binary variables representing spatial and momentum states.
- The constraints ensure the physical feasibility of the model, including terms for curvature, momentum, and angular momentum.

In [ ]:
def RHO(y, z):
    return np.sqrt(y**2 + z**2)
def dRS(x, rho, xs=2.5):
    return ((x - xs)**2 + rho**2)**(1/2)
def dFRS(rs, sigma=8, R=1):
    a = sigma * (np.tanh((R + rs)*sigma)**2 - 1)
    b = sigma * ((np.tanh(-(R - rs)*sigma)**2 - 1) / np.tanh(R * sigma))
    return (-1/2) * (a - b)
def THETA(x, p, xs=2.5, s=8, R=1):
    vs = R
    drs = dRS(x, p, xs)
    dfrs = dFRS(drs, s, R)
    return vs * ((x - xs) / drs) * dfrs
def CURVEConst(x, p, xs, a=2.5, b=1.5, c=3.0):
    return a * np.sin(x) + b * np.cos(p) + c * np.tan(xs)
def ENERGYConst(x, p, xs, alpha=1, beta=2, gamma=1.5):
    return alpha * x**2 + beta * p**2 + gamma * xs**2
def InterTERM(xV, pV, xsV, factor=0.5):
    return sum(factor * x * p * xs for x in xV for p in pV for xs in xsV)
def MOMENTUMConst(xV, pV, xsV, coef=1.0):
    return sum(coef * (xV[i] * pV[j] - xsV[k])**2
               for i in range(len(xV)) for j in range(len(pV)) for k in range(len(xsV)))
def AngularMomentumConst(xV, pV, xsV, coef=1.0):
    return sum(coef * (xV[i] * pV[j] + xsV[k])**2
               for i in range(len(xV)) for j in range(len(pV)) for k in range(len(xsV)))
def ENERGY(x, p, xs, coef=1.0):
    return coef * (np.sin(x) * np.cos(p) * np.tan(xs))
def BuildWarpDriveBQM(xm, xM, pm, pM, numPts, xsm, xsM, numSTEPS):
    xRan = np.linspace(xm, xM, numPts)
    pRan = np.linspace(pm, pM, numPts)
    xsRan = np.linspace(xsm, xsM, numSTEPS)
    xV = Array.create('x', shape=numPts, vartype='BINARY')
    pV = Array.create('p', shape=numPts, vartype='BINARY')
    xsV = Array.create('xs', shape=numSTEPS, vartype='BINARY')
    H = sum(THETA(xRan[i], pRan[j], xsRan[k]) * xV[i] * pV[j] * xsV[k]
            for i in range(numPts) for j in range(numPts) for k in range(numSTEPS))
    for var_array in [xV, pV, xsV]:
        H += 0.1 * Constraint((sum(var_array) - 1)**2, label='oneHotConst')
    CURVE = sum(CURVEConst(xRan[i], pRan[j], xsRan[k]) * xV[i] * pV[j] * xsV[k]
                     for i in range(numPts) for j in range(numPts) for k in range(numSTEPS))
    H += Constraint(CURVE, label='CURVEConst')
    INTER = InterTERM(xV, pV, xsV)
    H += Constraint(INTER, label='InterConst')
    MOMENTUM = MOMENTUMConst(xV, pV, xsV)
    H += Constraint(MOMENTUM, label='MOMENTUMConst')
    angMOMENTUM = AngularMomentumConst(xV, pV, xsV)
    H += Constraint(angMOMENTUM, label='AngularMomentumConst')
    EN = sum(ENERGY(xRan[i], pRan[j], xsRan[k]) * xV[i] * pV[j] * xsV[k]
                    for i in range(numPts) for j in range(numPts) for k in range(numSTEPS))
    H += Constraint(EN, label='ENERGY')
    model = H.compile()
    bqm = model.to_bqm()
    return bqm

def DeSolutionSH(xR, pR, xsR, solution):
    xSH = sum(xR[i] * solution.get(f'x[{i}]', 0) for i in range(len(xR)))
    pSH = sum(pR[i] * solution.get(f'p[{i}]', 0) for i in range(len(pR)))
    xsSH = sum(xsR[i] * solution.get(f'xs[{i}]', 0) for i in range(len(xsR)))
    return xSH, pSH, xsSH

def Visualization(solution):
    _, axs = plt.subplots(5, 2, figsize=(14, 30), subplot_kw={'projection': '3d'})
    axs = axs.flatten()
    x = np.linspace(xm, xM, num=numPts)
    p = np.linspace(pm, pM, num=numPts)
    xsVal = np.linspace(xsm, xsM, numSTEPS)
    xSH, pSH, xsSH = DeSolutionSH(x, p, xsVal, solution)
    xmADJ, xMADJ = xm + xSH, xM + xSH
    pmADJ, pMADJ = pm + pSH, pM + pSH
    xsmADJ, xsMADJ = xsm + xsSH, xsM + xsSH
    for ax in axs:
        ax.set_xlabel(r'$\phi_\mathrm{real}$')
        ax.set_ylabel(r'$\phi_\mathrm{im}$')
        ax.set_zlabel(r'$V(\phi)$')
        ax.set_xlim(xmADJ, xMADJ)
        ax.set_ylim(pmADJ, pMADJ)
        ax.set_zlim(-4.2, 4.2)
    X, P = np.meshgrid(np.linspace(xmADJ, xMADJ, num=numPts),
                       np.linspace(pmADJ, pMADJ, num=numPts))
    frame_count = 0
    for xs in np.linspace(xsmADJ, xsMADJ, numSTEPS):
        Z = THETA(X, P, xs, 8, 1)
        ax = axs[frame_count]
        ax.plot_wireframe(X, P, Z, rstride=2, cstride=2, linewidth=0.5, antialiased=True)
        ax.contour(X, P, Z, zdir='x', offset=xmADJ, cmap=plt.cm.coolwarm)
        ax.contour(X, P, Z, zdir='y', offset=pMADJ, cmap=plt.cm.coolwarm)
        ax.contour(X, P, Z, zdir='z', offset=-4.2, cmap=plt.cm.coolwarm)
        frame_count += 1
    plt.tight_layout()
    plt.show()

#### Build Alcubierre Warp Drive as BQM/QUBO

In [ ]:
bqm = BuildWarpDriveBQM(xm, xM, pm, pM, numPts, xsm, xsM, numSTEPS)

In [ ]:
#print(bqm)

#### COMPUTE on DYNEX

In [ ]:
model = dynex.BQM(bqm)
sampler = dynex.DynexSampler(model, mainnet=False, description='Quantum Alcubierre Warp Drive')
sampleset = sampler.sample(num_reads=100000, annealing_time = 500, debugging=False)
solution = sampleset.first.sample

#### Visualize the Warp Drive

In [ ]:
Visualization(solution)